# Day 18

## Part 1

Given a map like this:
```
#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################
```

Where `@` is us, `x` is a key opening a door `X`, `#` a wall, we need to find the shortest path to get all the keys.

In [1]:
def count_keys(map_):
    keys = 0
    for line in map_:
        for v in line:
            if v.islower():
                keys += 1
            
    return keys

In [2]:
def compute_path(map_):
    """Compute a path.
    
    A path is a list of tuples `(x, y, keys)`, `keys` being an ordered tuple
    of keys.
    At each intersections, it tries every path.
    
    Returns a list of tuples `(path, keys)`.
    """
    for y, line in enumerate(map_):
        for x, v in enumerate(line):
            if v == "@":
                path = [(x, y, [])]
                candidates = [path]

    result = []
    keys_count = count_keys(map_)

    while True:
        try:
            path = candidates.pop(0)
        except IndexError:
            return result

        px, py, keys = path[-1]
        for x, y in (px - 1, py), (px + 1, py), (px, py - 1), (px, py + 1):
            if (x, y, keys) in path:
                # We were already here we the same set of keys, this is a dead end,
                # let's stop.
                continue
            elif map_[y][x] == "#":
                continue
            elif map_[y][x].isupper():
                # It's door!
                key = map_[y][x].lower()
                if key in keys:
                    # We have the key, let's use it and go forward.
                    subpath = path + [(x, y, keys)]
                    candidates.append(subpath)
                else:
                    # We don't have this door's key, we can't go through it.
                    continue
            elif map_[y][x].islower() and map_[y][x] not in keys:
                # It's a key! Pick it up!
                subkeys = sorted(keys + [map_[y][x]])
                subpath = path + [(x, y, subkeys)]

                if len(subkeys) == keys_count:
                    # We found all the keys!
                    result.append(subpath)
                    continue

                candidates.append(subpath)
            else:
                # Empty path, let's go
                subpath = path + [(x, y, keys)]
                candidates.append(subpath)

In [3]:
def get_min_len(map_):
    map_ = [[v for v in line] for line in map_.splitlines() if line]
    result = compute_path(map_)
    keys_count = count_keys(map_)

    min_path = None
    for path in result:
        if len(path[-1][2]) != keys_count:
            continue

        if min_path is None:
            min_path = len(path)
        
    return min_path

In [4]:
%%time

get_min_len("""
########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################
""")  # 132

CPU times: user 24.3 ms, sys: 22 µs, total: 24.3 ms
Wall time: 24.5 ms


133

We get 133 because we can the origin.

In [5]:
%%time

get_min_len("""
#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################
""")

KeyboardInterrupt: 

Idée : on trie les chemins pour tester seulement les plus courts. Dès qu'on en a trouvé un, on enregistre sa taille et on drop tous les chemins plus grands.